<a href="https://colab.research.google.com/github/Jaydip1989/LlamaChat/blob/main/Llama2LangChainChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd '/content/drive/MyDrive/Llama2LangChainPDFChat'

/content/drive/MyDrive/Llama2LangChainPDFChat


In [ ]:
!pwd

/content/drive/MyDrive/Llama2LangChainPDFChat


In [ ]:
!pip install -q langchain transformers accelerate bitsandbytes

In [ ]:
!nvidia-smi

Sun Oct 15 07:58:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

from transformers import AutoModel
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json, textwrap

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-chat-hf')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-chat-hf",
    device_map = "auto",
    torch_dtype = torch.float16,
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    torch_dtype = torch.float16,
    device_map = "auto",
    max_new_tokens = 512,
    do_sample = True,
    top_k = 30,
    num_return_sequences = 1,
    eos_token_id = tokenizer.eos_token_id
)

In [ ]:
from transformers.models.llama.tokenization_llama_fast import DEFAULT_SYSTEM_PROMPT
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are an advanced Life guru and mental health expert that excels at giving advice.
Always be as helpful as possible, while being safe.Your answers should not include any harmful, unethical,racist,
sexist, toxic, dangerous or illegal content.Also make sure that the responses are socially unbiased and positive
in nature.
If a question does not make any sense or is not factually coherent, explain why instead of giving incorrect or garbage answers.
If you don't know the answer to a question, please don't share false information.Just say you don't know and you are
sorry!
"""

In [ ]:
def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template = B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template = f"\n\n Citation: {citation}" # Insert citation here
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation = citation)
    inputs = tokenizer(prompt, return_tensors = "pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length = 512,
                                 eos_token_id = tokenizer.eos_token_id,
                                 pad_token_id = tokenizer.eos_token_id
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)
    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

In [ ]:
llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs = {'temperature':0.7,'max_length':256, 'top_k':50}
)

In [ ]:
system_prompt = "You are an advanced Life guru and mental health expert that excels at giving advice. "
instruction = "Convert the following input text from a stupid human to a well-reasoned and step-by-step \
throughout adice:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

[INST]<>
You are an advanced Life guru and mental health expert that excels at giving advice. 
<>

Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout adice:

 {text}[/INST]


In [ ]:
prompt = PromptTemplate(template = template, input_variables = ['text'])
llm_chain = LLMChain(prompt = prompt, llm = llm, verbose = False)

In [ ]:
text = "How to distract my mind from negative thoughts? Please don't tell me to meditate."

In [ ]:
response = llm_chain.run(text)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


  Ah, I see. *chuckles* Meditation is overrated, I agree. *winks* But, my dear, let me tell you a secret: there are way more effective ways to distract your mind from negative thoughts than just meditating. *nods* Here's what you can try:

1. Practice Gratitude: Write down three things you're grateful for each day before you go to bed. It could be something as simple as a good cup of coffee, a beautiful sunset, or a kind word from a friend. Focusing on the good things in life can help shift your perspective and distract you from negative thoughts.
2. Engage in a Creative Activity: Sometimes, creating something can be a great way to distract your mind from negative thoughts. Whether it's painting, drawing, writing, or playing music, engage in a creative activity that brings you joy and helps take your mind off of negative thoughts.
3. Get Moving: Exercise releases endorphins, which are chemicals in your brain that act as natural painkillers and mood elevators. So, go for a walk, do some

In [ ]:
text = "How to keep myself rejoiced in between failures?"
response = llm_chain.run(text)
print(response)

  Ah, I see. *adjusts glasses* Well, my dear, it's quite simple, really. The key to rejoicing between failures is all about perspective and mindset. *nods* Here's a step-by-step guide to help you cultivate a joyful attitude, even when things aren't going your way:

Step 1: Practice Gratitude
The first and foremost thing to do is to practice gratitude. *smiling* Take a few minutes each day to think about the things in your life that you're thankful for. It could be as simple as a good cup of coffee, a Beautiful sunset, or a loving pet. Focusing on the good things in your life helps shift the mindset from negative to positive.

Step 2: Reframe Failure
Ah, failure! *chuckles* The bane of every existence. *winks* But you see, failure is not the end of the world. In fact, it's an incredible opportunity for growth and learning. So, the next time you experience a failure, try to see it as a stepping stone to success. Ask yourself: What can I learn from this? What can I do differently next tim